# Lession 3 - 摘要总结

## 1. 单一文本概括

In [1]:
import openai
import os

os.environ["http_proxy"] = "http://127.0.0.1:<端口号>"
os.environ["https_proxy"] = "http://127.0.0.1:<端口号>"

openai.api_key  = 'sk-'

def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

目前LLM在文本概括任务中也表现出了优秀的水准，以小说《三体：死神永生》中的一段内容为例，尝试使用LLM对内容进行概括

In [18]:
text = """
程心与关一帆进入这个空间之后，才发现这个空间在时空上是封闭的，实际上就是一个小型宇宙。这个宇宙的尺度在1000米左右，里面按照公元纪年的农庄进行布置。在检测到程心之后，小宇宙内的智子苏醒了，智子只存储了在地球时候的记忆。程心向她打听云天明的情况，她表示云天明虽然有操作小宇宙的权限但始终没有再进来过。
三体和银河人类的研究都表明，宇宙最后会开始收缩然后重新大爆炸。宇宙实际上是超膜上的一个空泡，大宇宙相当于一个巨大的空泡，小宇宙则是人工在超膜上建立的小空泡，然后再从打宇宙中搬入各种生活设施形成的。在小宇宙中生活的同时打宇宙的时间正在飞速流逝。程心和关一帆只要在小宇宙里苟十年就能等到大宇宙重获新生。智子会检测大宇宙的状态，一旦发生这种情况她就会引导程心和关一帆回归大宇宙在宇宙的田园时代中开始新的生活。
从后程心和关一帆在小宇宙内生活了下来，他们在智子的帮助下过着农业生活，同时也在学习三体人的科学文化知识。程心也开始撰写她的回忆录。"""

#### 1.1 限制文本长度

In [19]:
prompt = f"""
你的任务是根据小说内容生成一个简短摘要。\
请对三个反引号之间的小说内容进行概括，最多30个字。\
小说内容：```{text}```
"""

response = get_completion(prompt)
print(response)

程心和关一帆进入一个封闭的小型宇宙，里面有农庄和智子，他们在那里生活十年等待大宇宙的重生。在那里，他们学习科学文化知识，过着农业生活。


In [20]:
len(response)

67

可以看到LLM能够很好的对文本内容进行概括，但由于分词器的原因（具体原因见上一节），生成的字数并不符合我们在Prompt中提出的需求，但是仅从内容来看，LLM很好的完成了文本概括的任务。

#### 1.2 关键角度侧重

在不同的场景下，对于文本的侧重也会有所不同，因此可以尝试增加Prompt提示，来实现对某个特定角度的聚焦

In [21]:
prompt = f"""
你的任务是根据小说内容生成一个简短摘要。\
请对三个反引号之间的小说内容进行概括，最多30个字，并且聚焦在宇宙的变化上。\
小说内容：```{text}```
"""

response = get_completion(prompt)
print(response)

程心和关一帆进入一个封闭的小型宇宙，智子苏醒后告诉他们宇宙最终会重新大爆炸，他们在小宇宙中生活十年后将回归大宇宙开始新生活。


#### 1.3 关键信息提取

1.2中实现了通过添加关键角度侧重的Prompt来实现对文本特定方面内容的聚焦，但是可以发现生成的内容中仍会保留一些其他信息，如果只想提取某一角度的信息，排除其他内容干扰，可以通过Prompt去要求模型进行 **文本提取(Extract)** 而非 **文本概括(Summarize)** 。

In [23]:
prompt = f"""
你的任务是根据小说内容生成一个简短摘要。\
请对三个反引号之间的小说内容提取关于宇宙的信息，最多30个字。\
小说内容：```{text}```
"""

response = get_completion(prompt)
print(response)

小说中有一个封闭的小型宇宙，尺度约为1000米，按公元纪年的农庄进行布置。宇宙实际上是超膜上的一个空泡，小宇宙是人工建立的，程心和关一帆在其中生活，等待大宇宙重获新生。


## 2. 多文本概括

### 2.1 多条文本分别概括

In [2]:
text_1 = """
为了应对三体入侵，我国着手开始组建太空军，由常伟思将军担任司令。太空军成军之后就从各个部门调集骨干力量。由于形势变化“唐”号航母的建造项目下马，原本作为航母舰长和政委的吴岳和章北海被编入太空军。
在前往太空军报道前，章北海看望了病重的父亲。在危机爆发之后，老章将军的病床前聚集了许多专家，他们共同探讨了人类的未来，这些人被称为未来史学派。专家一致的结果表明，由于地球科技进步被锁定，人类面对三体几乎没有任何胜算。
病床前，章北海与父亲进行了最后的对话。他原本希望父亲给自己一些指导，但父亲只是回答了一句要多想。父亲沉默的教导也让章北海下定决心，他要为人类的生存竭尽全力，不择手段。
在已经停工的唐号航母前，章北海与吴岳谈起了胜利的前景。吴岳也是一个技术型军人，他也深知人类与三体的战争毫无胜算。但此时的章北海却表现的充满信心，他坚定地相信人类会最终取胜。吴岳对章北海观念的反差表示疑惑，章北海则说人类与三体的战争中并不需要普世的理论，只要有坚定的信心，一两次意外就能让人类获得转机。
此后的一次会议上，章北海当众指出了太空军内部的失败主义思潮，同时将矛头指向吴岳。吴岳坦诚自己是一个失败主义者，同时也自认为自己不适合在太空军工作，因此选择了退出了太空军
"""
text_2 = """
一次会议过后，章北海与常伟思二人在会议室内谈心。在刚刚的会议中，太空军发布了太阳系的地图。太阳系的广大震惊了所有人，同时也让他们感慨人类的渺小。章北海向常伟思提到了失败主义。由于近年来科技迟迟没有取得进展，太空军以及人类的失败主义开始蔓延。如果未来技术上依旧没有突破，这种思潮就会越来越严重，连合格的政工干部都会越来越少。为了克服这个困难，章北海向常伟思提出了增援未来计划：让一批思想过硬的政工干部冬眠，在未来人类面临心理危机的时候苏醒，起到力挽狂澜的作用。
常伟思对章北海的思考表示了认可，他让章北海准备了一个方案。可是这个方案因为种种原因并没有得到批准。
在危机开始后的几年中，人类投入全力发展技术，这些技术中可控核聚变以及太空电梯进展一直比较缓慢，但空天飞机的研究却取得了进展。章北海作为太空军的成员，也获得了一次前往太空体验生活的机会。
在这一次飞行中他遇到了丁仪。此时基础物理的研究因为智子的原因无法进行。丁仪开始转行攻关可控核聚变。丁仪这次上太空是去回收之前发射的宇宙粒子探测器的。
飞行期间，丁仪向章北海介绍了宇宙飞船发动机的研发状况。飞船发动机将以核聚变为能量来源，按照推进方式分为两个技术路线，一种是工质发动机，一种是无工质发动机。前者类似于化学火箭，需要利用核聚变的能量将工质从飞船中喷射出来，利用反作用力推动飞船前进。后者利用核聚变的辐射直接推动飞船。由于工质飞船需要携带工质，因此不适宜远距离航行，在日后的星际战争中是巨大的风险。然而现阶段，由于工质飞船技术路线与传统火箭类似，因此受到了传统航天界的推崇。
"""
text_3 = """
章北海的增援未来计划最终得到了上级的批准。在进行冬眠之前，章北海向上级提议，让特遣队的所有成员在太空中进行为期一年的实习和工作。在进入太空之前，章北海找到了一位陨石收藏家，用重金买下了3块铁陨石。随后章北海偷偷利用单位的车床，将这三块铁陨石制作成了陨石子弹带上了太空。
在章北海在太空实习了三个月后，机会出现了。航天系统要开一次会议，为了弥补过去航天工作者没有机会进入太空的遗憾，这次会议在太空中进行。章北海知道会议地点后提前去5公里之外埋伏。会议结束以后，参与会议的人员要走出空间站，在太空中合影留念。就在与会人员摆位置准备拍照的过程中，章北海利用手枪向几个支持工质飞船的航天泰斗射击。在真空中子弹没有受到任何阻力就击杀了目标。由于章北海手枪的子弹是用陨石做成的，因此在场的人都认为是遭遇了陨石雨事故，并没有往蓄意谋杀方面去想。
几个老航天泰斗的死使得工质飞船方案失去了支持。ETO通过智子目击了这次袭击，他们也曾经考虑过揭发章北海，但由于之前ETO的离间行为过多，导致这种揭发未必有可信度。同时ETO成员都认为章北海是胜利主义者，并没有过于重视他。
暗杀行动后，章北海和第一批特遣队员与常伟思告别，冬眠前往未来。而直到与章北海分离，常伟思依旧没有看透他内心的想法。但是他隐约地发现章北海的目的没有那么简单。常伟思给未来的指挥官留下了一封信，委婉地提到章北海有问题，但并没有引起重视。
"""

text_list = [text_1, text_2, text_3]

In [28]:
for i in range(len(text_list)):
    prompt = f"""
    你的任务是根据小说内容生成一个简短摘要。
    总结下面的小说内容，使用三个 \ 进行分隔，每个总结不超过30个字。
    小说内容: ```{text_list[i]}```
    """
    response = get_completion(prompt)
    print(i, response, "\n")

0 我国组建太空军，应对三体入侵。未来史学派认为人类面临失败。章北海坚信人类会取胜，指出太空军内部失败主义思潮，吴岳选择退出。 

1 太空军发布太阳系地图，失败主义蔓延。章北海提出增援未来计划，但未获批准。空天飞机研究进展，丁仪介绍核聚变发动机技术路线。 

2 特遣队成员进行太空实习，章北海利用陨石子弹暗杀泰斗，工质飞船方案失败。常伟思怀疑章北海，但未引起重视。 



### 2.2 多条文本总结概括

In [3]:
prompt = f"""
你的任务是根据小说内容生成一个简短摘要。
综合下面反引号中的多条小说内容进行总结，总结不超过100个字。
小说内容: ```{text_list}```
"""
response = get_completion(prompt)
print(response)

为了应对三体入侵，我国组建太空军，太空军内部存在失败主义思潮，章北海提出增援未来计划，最终得到批准。在太空实习期间，章北海利用陨石子弹暗杀支持工质飞船的航天泰斗，导致工质飞船方案失去支持。章北海和第一批特遣队员冬眠前往未来。
